In [38]:
import pandas as pd
import os
import vobject
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

directory = './'

In [43]:
# Read vcard file line-by-line. 
# Check for FN value and add if not in group (FN values is often required)
# create dictionary to count number of values of each type 
# MIME Unwrap multi-line entries into a single line (multi line values don't seem to sync with iPhone)

fileIn = open('iCloud 20210106.vcf', 'r') 
Lines = fileIn.readlines() 
fileIn.close()

fileOut = open('iCloudFix 20210106.vcf', 'w') 

lineInCount = 0 
lineOutCount = 0 
recOutCount = 0
wrappedLineInCount = 0
cardCount = 0
addrCount = 0 
telCount = 0
emailCount = 0
fnCount = 0
outCount = 0 

recsOut = []
types = {}

for lineIn in Lines: 
    lineInCount += 1
    # MIME Unwrap
    if (ord(lineIn[0:1]) == 0x09) | (ord(lineIn[0:1]) == 0x20):
        linesOut[len(linesOut)-1] = linesOut[len(linesOut)-1][0:-1] + lineIn[1:len(lineIn)]
        wrappedLineInCount += 1
        # print(linesOut)
    else:
        
        if lineIn == 'BEGIN:VCARD\n':
            linesOut = []
            linesOut.append(lineIn)  
            cardCount += 1
            foundFN = False
            saveEmail = ""
            saveFN = ""
            saveN = ""

        elif lineIn == 'END:VCARD\n':
            if not foundFN:
                linesOut.append('FN: Dummy Value\n')
                print("No FN record found at line {}, record {} \nsample: {} \n{}\n{}".format(lineInCount, cardCount, saveEmail, saveFN, saveN))
            linesOut.append(lineIn)  
            outCount += 1
    #        if (outCount >= 600) & (outCount < 700):
            lineOutCount += len(linesOut)
            recOutCount += 1
            fileOut.writelines(linesOut)          


        else:
            if (lineIn.find(";") < lineIn.find(":")) &  (lineIn.find(";") > 0):
                typeCut = lineIn.find(";")
            else:
                typeCut = lineIn.find(":")

            if (typeCut > 0):
                type = lineIn[0:typeCut]
                if type in types:
                    types[type] += 1
                else:
                    types[type] = 1

            if lineIn[0:3] == 'FN:':
                fnCount += 1
                foundFN = True

  

            linesOut.append(lineIn)


fileOut.close()


print("Read:  \n  {} total lines ({} wrapped)\n  {}  total records\nWrote: \n  {} lines \n  {}  Total Records \n\n"\
      .format(lineInCount, wrappedLineInCount, cardCount, lineOutCount, recOutCount))
print("Count by type:  \n", types)

#8660 total lines 
#651 total records with 276 addresses

#iCloud2:  {'EMAIL': 565, 'TEL': 708, 'ADR': 309, 'item1.URL': 20, 'BDAY': 4, 'item1.X-ABRELATEDNAMES': 28, 'item2.X-ABRELATEDNAMES': 2, 'item2.URL': 1}


Read:  
  7559 total lines (329 wrapped)
  670 total records
Wrote: 
  7230 lines 
  670 Total Records 


Count by type:  
 {'VERSION': 670, 'PRODID': 670, 'N': 670, 'FN': 670, 'ORG': 448, 'TITLE': 306, 'EMAIL': 583, 'TEL': 730, 'ADR': 325, 'NOTE': 438, 'item1.URL': 20, 'item1.X-ABLabel': 69, 'BDAY': 15, 'REV': 217, 'item1.TEL': 9, 'item2.X-ABRELATEDNAMES': 3, 'item2.X-ABLabel': 4, 'X-ABShowAs': 2, 'item1.X-ABRELATEDNAMES': 40, 'item2.URL': 1}


In [33]:
fileOut.close()